In [1]:
import os
os.chdir("..")

In [16]:
import json
import numpy as np
import math
from typing import Dict, List, Tuple
import copy
from itertools import combinations
from collections import Counter

import logging
from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector, ConflictPredicator

root_folder = os.path.join(DATA_ROOT, THERMO_ROOT)

test_projects = [
    "HS3301",
    # "HS3304",
    # "HS3307",
    # "HS3308",
    # "HS3309",
    # "HS3310",
    # "HS3311",
    "HS3313",
    # "HS5302",
    # "HS5303",
    # "HS5304",
    # "HS5305",
    "HS5309",
    "HS5314",
]
all_setpoints = [18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5]
setpoint_mapping = {
    "thermostat": {"off": 0, 0: "off"}
}
for idx, point in enumerate(all_setpoints):
    state_full_name = "auto" + DEVICE_MULTI_STATE_SUFFIX + str(point)
    setpoint_mapping["thermostat"][state_full_name] = idx + 1
    setpoint_mapping["thermostat"][idx + 1] = state_full_name
ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
capacity = {
    "thermostat":0
}
BOOL_SIM = False
BOOL_UMASS= False
ccp_alpha = 1e-5
test_dates = generate_test_date(root_folder, test_projects, test_ratio = 0.4, true_random=False, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)

In [17]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], grid_data = test_umass(root_folder, test_project=p, ctx_info=ctx_info, ccp_alpha=ccp_alpha, test_dates=test_dates, is_sim=BOOL_SIM, is_umass=BOOL_UMASS, d_mapping=setpoint_mapping)

c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print("Final predicted conflicts" + str({x:len(final_conflicts[x]) for x in final_conflicts}))

Mined pattern{'thermostat': 181}
Mined pattern{'thermostat': 199}
Mined pattern{'thermostat': 167}
Mined pattern{'thermostat': 207}
Final predicted conflicts{'thermostat': 583}


In [23]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))
        # for g in d_groups:
        #     print(ctx_info.coor_box_to_range(g["box"][0] + g["box"][1]))
        #     print(g["dis"])

probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["thermostat"])]
print(probs_i)


The habit groups found in HS3301
thermostat 81
The habit groups found in HS3313
thermostat 68
The habit groups found in HS5309
thermostat 72
The habit groups found in HS5314
thermostat 65
[(0.8936735778841043, 0), (0.8747484473232688, 1), (0.8687416975985931, 2), (0.9073901120021759, 3), (0.936992270198474, 4), (0.9033556438907993, 5), (0.906002989864388, 6), (0.9318097843856028, 7), (0.9168187327704391, 8), (0.882654649765587, 9), (0.845380908231069, 10), (0.884701492635308, 11), (0.8818995070171254, 12), (0.843720228407759, 13), (0.8359854576350849, 14), (0.893303475837139, 15), (0.8093013864475629, 16), (0.8277068950091283, 17), (0.8325240725245282, 18), (0.8018750912009338, 19), (0.7600139137675895, 20), (0.6937303716644828, 21), (0.8413605565533757, 22), (0.7064796905222437, 23), (0.891480277766563, 24), (0.8755204202289275, 25), (0.893903251056967, 26), (0.9131695926008261, 27), (0.8773787631816294, 28), (0.9485245700245699, 29), (0.9071011821758088, 30), (0.9447504621770987, 31)

In [19]:
import copy
# Make ground truth:

gt_ctx_info = ContextAccessor({
        TIME_CTX: {
            "range" : (0, 24*60),
            "interval" : 60,
        },
        "OutTemp#NUM": {
            "range": (-25, 35),
            "interval": 5,
        },
        # WEEKDAY_CTX: {
        #     "range": (0, 6),
        #     "interval": 1,
        # },
})
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(root_folder, p, is_sim=BOOL_SIM, is_umass=BOOL_UMASS)
    device_events[p] = device_evts
gtconflict_cfg = {
    "context_info": gt_ctx_info,
    "capacity": capacity
}
# test_dates = {}

conflict_finder = GtConflictFinder(gtconflict_cfg)
gt_conflicts, test_state_cnt = conflict_finder.get_Gt_conflict(ctx_evts, device_events, test_dates)
print(len(gt_conflicts))

4833


In [24]:
conflict_device = {
    d:[]
    for d in capacity
}

conflict_state_device = {
    d:{}
    for d in capacity
}
MIN_TEST_OBS = 19
cnts = []
for c in gt_conflicts:
    d = c["device"]
    conflict_device[d].append(c)
    s = gt_ctx_info.get_coor_by_ctx(c["ctx"])
    users = frozenset(c["device_states"].keys())
    cnts.append(test_state_cnt[d][users][s])
    if test_state_cnt[d][users][s] < MIN_TEST_OBS:
        continue
    if s not in conflict_state_device[d]:
        conflict_state_device[d][s] = {}
    conflict_state_device[d][s][users] = conflict_state_device[d][s].get(users, 0) + 1

    # if conflict_state_device[d][s][users] > test_state_cnt[d][s]:
    #     print(conflict_state_device[d][s][users], test_state_cnt[d][s])
    #     print(users, d, c["cur_time"], s)
    
print({d:len(c) for d,c in conflict_device.items()})
print({d:len(c) for d,c in conflict_state_device.items()})
print(len(test_dates))
print(Counter(cnts))

{'thermostat': 4833}
{'thermostat': 82}
145
Counter({15.0: 361, 9.0: 349, 10.0: 322, 14.0: 310, 5.0: 297, 4.0: 291, 11.0: 283, 8.0: 277, 3.0: 275, 12.0: 268, 7.0: 260, 16.0: 251, 13.0: 244, 2.0: 230, 6.0: 196, 18.0: 172, 17.0: 121, 1.0: 114, 19.0: 84, 20.0: 36, 21.0: 29, 23.0: 26, 24.0: 23, 22.0: 14})


In [25]:
# print(conflict_state_device["thermostat"])

In [26]:
all_users = test_projects
all_devices = capacity.keys()
user_pairs = list(combinations(all_users, 2))
conflict_predicator = ConflictPredicator(ctx_info, final_conflicts)
exp_cnt = 0
exp_gt_c = 0
exp_result = {d:[0,0,0,0,0] for d in all_devices}
max_p = 0
all_state_cnt = 0
for d in all_devices:
    for u_pair in user_pairs:
        it = np.nditer(test_state_cnt[d][frozenset(u_pair)], flags=['multi_index'])
        for count in it:
            all_state_cnt += count
            if count < MIN_TEST_OBS:
                continue
            state = it.multi_index
            gt_prob = 0.
            if (d in conflict_state_device) and \
                (state in conflict_state_device[d]) and \
                (frozenset(u_pair) in conflict_state_device[d][state]):

                gt_prob =  float(conflict_state_device[d][state][frozenset(u_pair)]) / count
            gt_ctx_snapshot = gt_ctx_info.coor_to_snapshot(state)

            pred_prob = conflict_predicator.get_prob_conflict(ctx_info.get_coor_by_ctx(gt_ctx_snapshot), u_pair, d)
            exp_cnt += 1
            if gt_prob > 0:
                # if gt_prob > 0.5:
                #     print(gt_ctx_info.coor_to_snapshot(state).values(), gt_prob, pred_prob, u_pair)
                exp_gt_c += 1
                exp_result[d][0] += abs(pred_prob - gt_prob)
                exp_result[d][1] += 1
                exp_result[d][4] += gt_prob
                if gt_prob > 1:
                    print(gt_prob, pred_prob, d, u_pair)
                # print(pred_prob, gt_prob, gt_ctx_snapshot, count)
                max_p = max(gt_prob, max_p)

            else:
                exp_result[d][2] += pred_prob - gt_prob
                exp_result[d][3] += 1

for d in exp_result:
    exp_result[d].append((exp_result[d][0] + exp_result[d][2]) / (exp_result[d][1] + exp_result[d][3]))
    if exp_result[d][1] > 0:
        exp_result[d][0] = exp_result[d][0] / exp_result[d][1]
        exp_result[d][4] = exp_result[d][4] / exp_result[d][1]
    if exp_result[d][3] > 0:
        exp_result[d][2] = exp_result[d][2] / exp_result[d][3]
    
print(max_p)
print(exp_result)
print(all_state_cnt)

1.0
{'thermostat': [0.14579152288229472, 332, 0.5302174080865139, 1, 0.6828463186637194, 0.14694595496999507]}
6924.0
